# Malaria_images_DL

Model to classify whether the cell image is Maleria infacted or Uninfected
## The Data

The dataset contains 2 folders - Infected - Uninfected

And a total of 27,558 images.

In [2]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.image import imread
from glob import glob
import tensorflow as tf
import cv2
import pathlib

from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator

## Data

In [3]:
image_data_directory = '../input/malerial-cell-classification-dataset/cell_images/cell_images'
data_dir = pathlib.Path(image_data_directory)
data_dir

In [4]:
print("Number of persitized images  =", len(os.listdir(image_data_directory + "/Parasitized/")))
print("Number of Uninfected images  =", len(os.listdir(image_data_directory + "/Uninfected/")))

In [5]:
os.listdir(image_data_directory + "/Parasitized/")[0:10]

In [6]:
os.listdir(image_data_directory + "/Uninfected/")[0:10]

In [7]:
plt.figure(figsize = (14, 7))
plt.imshow(cv2.imread('../input/malerial-cell-classification-dataset/cell_images/cell_images/Parasitized/C99P60ThinF_IMG_20150918_141001_cell_93.png'))
plt.title('Parasitized Cell')
plt.show()


In [8]:
plt.figure(figsize = (14, 7))
plt.imshow(cv2.imread('../input/malerial-cell-classification-dataset/cell_images/cell_images/Uninfected/C203ThinF_IMG_20151029_102123_cell_51.png'))
plt.title('Uninfected Cell')
plt.show()


In [9]:
train_gen=ImageDataGenerator(rescale=1./255,validation_split=0.3)

train_data=train_gen.flow_from_directory(
    data_dir,
    target_size=(128,128),
    #labels='inferred',
    batch_size=32,
    seed=64,
    color_mode='rgb',
    class_mode='binary',
    shuffle=True,
subset='training')

test_data=train_gen.flow_from_directory(
    data_dir,
    target_size=(128,128),
    #labels='inferred',
    batch_size=32,
    seed=64,
    color_mode='rgb',
    class_mode='binary',
    shuffle=True,
subset='validation')

In [10]:
train_data

In [11]:
test_data

In [12]:
train_data.class_indices, train_data.classes

In [13]:
test_data.class_indices, test_data.classes

## Model

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras. callbacks import EarlyStopping

In [15]:
model = Sequential()

# Convoluntional Layers
model.add(Conv2D(filters=32, 
                 kernel_size=(3,3), 
                 input_shape= (128, 128, 3),
                 activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=64, 
                 kernel_size=(3,3), 
                 input_shape=(128, 128, 3),
                 activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=64, 
                 kernel_size=(3,3), 
                 input_shape=(128, 128, 3), 
                 activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))

# flatten out the layers
model.add(Flatten())

# Dense layer

model.add(Dense(128, activation='relu'))

# Dropout layer
model.add(Dropout(0.5))


# Output lyaer
model.add(Dense(1, activation='sigmoid'))

# compile the model
model.compile(loss='binary_crossentropy', optimizer='adam',
                         metrics=['accuracy'])

#### Model Summary

In [16]:
model.summary()

### Early Stopping

In [17]:
early_stopping = EarlyStopping(monitor='val_loss', patience=2, verbose=1)

# <a name='training'>Training the Model</a>

### Actual Training

In [18]:
tf.get_logger().setLevel("INFO")
results = model.fit(train_data, 
                            validation_data=test_data,
                            callbacks=[early_stopping],
                            epochs=20,
                            verbose=1)

--------

## Evaluating the Model

## Model Metrics and Evaluate

In [19]:
model.metrics_names

In [21]:
model.evaluate(test_data)

## Prediction

### Prediction Probabilities

In [23]:
pred_probabilities = model.predict(test_data)

In [24]:
pred_probabilities

In [25]:
predictions = pred_probabilities > 0.5 # can change this number based on our decision

In [26]:
len(predictions)

In [27]:
predictions